# Menganalisis Risiko Gagal Bayar Peminjam

Proyek ini bertujuan untuk mengetahui pengaruh status perkawinan seorang nasabah dan jumlah anak yang dimilikinya terhadap probabilitas gagal bayar dalam pelunasan pinjaman. Laporan ini nantinya akan dipertimbangkan pada saat membuat **penilaian kredit** untuk calon nasabah. **Penilaian kredit** digunakan untuk mengevaluasi kemampuan calon peminjam untuk melunasi pinjaman mereka.

# Konten <a id='back'></a>

* [Tahap 1. Ikhtisar Data](#data_review)
* [Tahap 2. Pra-pemrosesan data](#data_preprocessing)
    * [2.1 Transformasi data](#transformation)
    * [2.2 Bekerja dengan nilai yang Hilang](#missing_values)
    * [2.3 Pengkategorian data](#categorize)
    * [2.4 Kesimpulan](#data_preprocessing_conclusions)
* [Tahap 3. Pengujian Hipotesis](#hypotheses)
* [Kesimpulan](#end)

## Tahap 1: Ikhtisar data


In [1]:
import pandas as pd
df = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [4]:
# Mari kita lihat informasi umum dari dataset kita
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Dari info yang diperoleh dari dataset, diketahui bahwa terdapat nilai yang hilang di kolom `days_employed` dan `total_income` dengan jumlah yang sama.

In [5]:
# Mari tampilkan 10 baris pertama
df.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Dari data di atas terlihat bahwa nilai `days_employed` seharusnya positif dan terdapat nilai minimal pada `children` yaitu minus 1 dan maksimal 20 yang dirasa tidak masuk akal

In [7]:
# Mari kita lihat tabel yang telah difilter dengan nilai yang hilang di kolom pertama yang mengandung data yang hilang
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [8]:
df[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [9]:
# Mari kita terapkan beberapa kondisi untuk memfilter data dan melihat jumlah baris dalam tabel yang telah difilter.
df[(df['days_employed'].isna()) & (df['total_income'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Kesimpulan sementara**

Setelah memfilter data dari kolom `days_employed` dan `total_income`, dapat diketahui bahwa nilai yang hilang dari kedua kolom simetris. Kedua kolom memiliki nilai null sebanyak 2174 baris. Langkah selanjutnya yaitu menghitung presentase dari data yang hilang untuk menentukan apakah data tersebut bisa dihilangkan atau tidak.

In [10]:
# Mari kita periksa nasabah yang tidak memiliki data tentang karakteristik yang teridentifikasi dan kolom dengan nilai yang hilang
df_na = df[(df['days_employed'].isna()) & (df['total_income'].isna())]
df_na.reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
1,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
2,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
3,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
4,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
2169,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
2170,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
2171,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
2172,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [11]:
#memeriksa distribusinya
a = df_na['income_type'].value_counts(normalize=True).reset_index().rename(columns={"income_type":"percentage"})
a['percentage'] = a['percentage'].apply("{:,.2%}".format)
b = df_na['income_type'].value_counts().reset_index().rename(columns={"income_type":"count"})
c = pd.concat([a, b[['count']]], axis=1)
c

,index,percentage,count
0,employee,50.83%,1105
1,business,23.37%,508
2,retiree,19.00%,413
3,civil servant,6.76%,147
4,entrepreneur,0.05%,1


Nilai yang hilang pada kolom `income_type` tidak memiliki pola tertentu karena terdapat berbagai nilai yang hilang dengan nilai yang tertinggi yaitu employee sebesar 50 persen.

In [12]:
# Memeriksa distribusi di seluruh *dataset*
a1 = df['income_type'].value_counts(normalize=True).reset_index().rename(columns={"income_type":"percentage"})
a1['percentage'] = a1['percentage'].apply("{:,.2%}".format)
b1 = df['income_type'].value_counts().reset_index().rename(columns={"income_type":"count"})
c1 = pd.concat([a1, b1[['count']]], axis=1)
c1

,index,percentage,count
0,employee,51.66%,11119
1,business,23.62%,5085
2,retiree,17.91%,3856
3,civil servant,6.78%,1459
4,entrepreneur,0.01%,2
5,unemployed,0.01%,2
6,student,0.00%,1
7,paternity / maternity leave,0.00%,1


**Kesimpulan sementara**

Distribusi keseluruhan data `income_type` terhadap nilainya yang hilang memiliki kemiripan. Hal ini menandakan tidak adanya pola tertentu. Selanjutnya kita akan mencari tahu bagaimana dengan distribusi data di kolom lain.

In [13]:
a2 = df_na['dob_years'].value_counts(normalize=True).reset_index().rename(columns={"dob_years":"percentage"})
a2['percentage'] = a2['percentage'].apply("{:,.2%}".format)
b2 = df_na['dob_years'].value_counts().reset_index().rename(columns={"dob_years":"count"})
c2 = pd.concat([a2, b2[['count']]], axis=1)
c2

,index,percentage,count
0,34,3.17%,69
1,40,3.04%,66
2,31,2.99%,65
3,42,2.99%,65
4,35,2.94%,64
5,36,2.90%,63
6,47,2.71%,59
7,41,2.71%,59
8,30,2.67%,58
9,28,2.62%,57


In [14]:
a3 = df['dob_years'].value_counts(normalize=True).reset_index().rename(columns={"dob_years":"percentage"})
a3['percentage'] = a3['percentage'].apply("{:,.2%}".format)
b3 = df['dob_years'].value_counts().reset_index().rename(columns={"dob_years":"count"})
c3 = pd.concat([a3, b3[['count']]], axis=1)
c3 

,index,percentage,count
0,35,2.87%,617
1,40,2.83%,609
2,41,2.82%,607
3,34,2.80%,603
4,38,2.78%,598
5,42,2.77%,597
6,33,2.70%,581
7,39,2.66%,573
8,31,2.60%,560
9,36,2.58%,555


In [15]:
# Periksa penyebab dan pola lain yang dapat mengakibatkan nilai yang hilang
a4 = df_na['family_status'].value_counts(normalize=True).reset_index().rename(columns={"family_status":"percentage"})
a4['percentage'] = a4['percentage'].apply("{:,.2%}".format)
b4 = df_na['family_status'].value_counts().reset_index().rename(columns={"family_status":"count"})
c4 = pd.concat([a4, b4[['count']]], axis=1)
c4

,index,percentage,count
0,married,56.90%,1237
1,civil partnership,20.33%,442
2,unmarried,13.25%,288
3,divorced,5.15%,112
4,widow / widower,4.37%,95


In [16]:
a5 = df['family_status'].value_counts(normalize=True).reset_index().rename(columns={"family_status":"percentage"})
a5['percentage'] = a5['percentage'].apply("{:,.2%}".format)
b5 = df['family_status'].value_counts().reset_index().rename(columns={"family_status":"count"})
c5 = pd.concat([a5, b5[['count']]], axis=1)
c5

,index,percentage,count
0,married,57.51%,12380
1,civil partnership,19.41%,4177
2,unmarried,13.07%,2813
3,divorced,5.55%,1195
4,widow / widower,4.46%,960


**Kesimpulan sementara**

Pada kolom `dob_years` dan `family_status` juga tidak memiliki pola karena ada berbagai macam nilai yang hilang dan persebaran datanya memiliki kemiripan dengan dataset keseluruhan.

In [17]:
# Periksa pola lainnya - jelaskan pola tersebut
a6 = df_na['education'].value_counts(normalize=True).reset_index().rename(columns={"education":"percentage"})
a6['percentage'] = a6['percentage'].apply("{:,.2%}".format)
b6 = df_na['education'].value_counts().reset_index().rename(columns={"education":"count"})
c6 = pd.concat([a6, b6[['count']]], axis=1)
c6

,index,percentage,count
0,secondary education,64.77%,1408
1,bachelor's degree,22.82%,496
2,SECONDARY EDUCATION,3.08%,67
3,Secondary Education,2.99%,65
4,some college,2.53%,55
5,Bachelor's Degree,1.15%,25
6,BACHELOR'S DEGREE,1.06%,23
7,primary education,0.87%,19
8,SOME COLLEGE,0.32%,7
9,Some College,0.32%,7


Pada kolom `education` terlihat adanya nilai duplikat sehingga perlu diperbaiki nantinya.

In [18]:
a7 = df_na['children'].value_counts(normalize=True).reset_index().rename(columns={"children":"percentage"})
a7['percentage'] = a7['percentage'].apply("{:,.2%}".format)
b7 = df_na['children'].value_counts().reset_index().rename(columns={"children":"count"})
c7 = pd.concat([a7, b7[['count']]], axis=1)
c7

,index,percentage,count
0,0,66.19%,1439
1,1,21.85%,475
2,2,9.38%,204
3,3,1.66%,36
4,20,0.41%,9
5,4,0.32%,7
6,-1,0.14%,3
7,5,0.05%,1


Pada kolom `children` terlihat adanya nilai -1 dan 20 yang dirasa tidak mungkin sehingga perlu diperbaiki nantinya.

In [19]:
a7 = df_na['gender'].value_counts(normalize=True).reset_index().rename(columns={"gender":"percentage"})
a7['percentage'] = a7['percentage'].apply("{:,.2%}".format)
b7 = df_na['gender'].value_counts().reset_index().rename(columns={"gender":"count"})
c7 = pd.concat([a7, b7[['count']]], axis=1)
c7

,index,percentage,count
0,F,68.26%,1484
1,M,31.74%,690


In [20]:
a8 = df_na['purpose'].value_counts(normalize=True).reset_index().rename(columns={"purpose":"percentage"})
a8['percentage'] = a8['percentage'].apply("{:,.2%}".format)
b8 = df_na['purpose'].value_counts().reset_index().rename(columns={"purpose":"count"})
c8 = pd.concat([a8, b8[['count']]], axis=1)
c8

,index,percentage,count
0,having a wedding,4.23%,92
1,to have a wedding,3.73%,81
2,wedding ceremony,3.50%,76
3,construction of own property,3.45%,75
4,housing transactions,3.40%,74
5,buy real estate,3.31%,72
6,purchase of the house for my family,3.27%,71
7,transactions with my real estate,3.27%,71
8,housing renovation,3.22%,70
9,transactions with commercial real estate,3.22%,70


**Kesimpulan**

Setelah memeriksa semua kolom, tidak ditemukan suatu pola tertentu. Untuk mengatasi permasalahan ini yaitu dengan menghilangkan nilai duplikat yang ada, merapikan penulisan besar kecilnya, mengganti tipe data dan merubah nilai yang negatif lalu mengatasi nilai yang hilang dengan menggunakan mean/median dari data.

## Tahap 2 : Pra-pemrosesan data

### Transformasi data

In [21]:
# Mari kita lihat semua nilai di kolom pendidikan untuk memeriksa ejaan apa yang perlu diperbaiki
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [22]:
# Perbaiki pencatatan jika diperlukan
df['education'] = df['education'].str.lower()

In [23]:
# Periksa semua nilai di kolom untuk memastikan bahwa kita telah memperbaikinya dengan tepat
#cek lagi
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

Selanjutnya kita akan memeriksa data kolom `children`

In [24]:
# Mari kita lihat distribusi nilai pada kolom `children`
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Terdapat nilai minus dan nilai 20 pada jumlah anak yang rasanya tidak logis. Hal ini dapat disebabkan oleh kesalahan penginputan data. Oleh karena itu, kita harus memperbaikinya

In [25]:
df[df['children'] < 0].count()

children            47
days_employed       44
dob_years           47
education           47
education_id        47
family_status       47
family_status_id    47
gender              47
income_type         47
debt                47
total_income        44
purpose             47
dtype: int64

In [26]:
df[df['children'] > 5].count()

children            76
days_employed       67
dob_years           76
education           76
education_id        76
family_status       76
family_status_id    76
gender              76
income_type         76
debt                76
total_income        67
purpose             76
dtype: int64

Karena data yang salah hanya sedikit jika dibandingkan dengan keseluruhan data yang kemungkinan disebabkan oleh kesalahan penginputan, maka untuk nilai yang -1 akan dijadikan 1 serta nilai 20 akan dijadikan 2.

In [27]:
# [perbaiki data berdasarkan keputusan Anda]
df['children'] = df['children'].replace(20,2)

In [28]:
# Periksa kembali kolom `children` untuk memastikan bahwa semuanya telah diperbaiki
df['children'].value_counts()

 0    14149
 1     4818
 2     2131
 3      330
-1       47
 4       41
 5        9
Name: children, dtype: int64

In [29]:
# Temukan data yang bermasalah di kolom `days_employed` jika memang terdapat masalah, dan hitung persentasenya
df['days_employed']

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [30]:
# Atasi nilai yang bermasalah, jika ada
df['days_employed'] = df['days_employed'].abs()


In [31]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
df['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [32]:
# Periksa `dob_years` untuk nilai yang mencurigakan dan hitung persentasenya
df['dob_years'].value_counts(normalize=True)

35    0.028664
40    0.028293
41    0.028200
34    0.028014
38    0.027782
42    0.027735
33    0.026992
39    0.026620
31    0.026016
36    0.025784
44    0.025412
29    0.025319
30    0.025087
48    0.024994
37    0.024948
50    0.023879
43    0.023833
32    0.023693
49    0.023600
28    0.023368
45    0.023089
27    0.022904
56    0.022625
52    0.022485
47    0.022300
54    0.022253
46    0.022067
58    0.021417
57    0.021370
53    0.021324
51    0.020813
59    0.020627
55    0.020581
26    0.018955
60    0.017515
25    0.016585
61    0.016492
62    0.016353
63    0.012497
64    0.012311
24    0.012265
23    0.011800
65    0.009013
66    0.008502
22    0.008502
67    0.007758
21    0.005157
0     0.004692
68    0.004599
69    0.003949
70    0.003020
71    0.002695
20    0.002369
72    0.001533
19    0.000650
73    0.000372
74    0.000279
75    0.000046
Name: dob_years, dtype: float64

Terdapat nilai 0 pada `dob_years`. Sulit untuk mengetahui apakah data ini disebabkan oleh kesalahan input atau lainnya. Oleh karena itu, data yang usianya 0 lebih baik tidak dimasukkan ke dataset.

In [33]:
# Atasi masalah pada kolom `dob_years`, jika terdapat masalah
zero_dob = df[df['dob_years'] == 0]
zero_dob

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,secondary education,1,married,0,F,retiree,0,11406.644,car
149,0,2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
270,3,1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0,397856.565013,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1040,0,1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,secondary education,1,married,0,F,employee,0,NaN,housing
20462,0,338734.868540,0,secondary education,1,married,0,F,retiree,0,41471.027,purchase of my own house
20577,0,331741.271455,0,secondary education,1,unmarried,4,F,retiree,0,20766.202,property
21179,2,108.967042,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate


In [34]:
percent_zero_dob = zero_dob['dob_years'].count() / df['dob_years'].count()
print(percent_zero_dob )

0.004692218350754936


Karena presentasinya kecil, maka data yang dihapus tidak akan berpengaruh besar terhadap dataset. Selanjutnya kita akan membuat dataset baru yang tidak mengikutsertakan `dob_years` bernilai 0.

In [35]:
df_new = df
df_new = df_new[df_new['dob_years'] != 0]

In [36]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
df_new['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

In [37]:
# Mari kita lihat nilai untuk kolom ini
df_new['family_status'].value_counts()

married              12331
civil partnership     4156
unmarried             2797
divorced              1185
widow / widower        955
Name: family_status, dtype: int64

In [38]:
# Mari kita liat nilai dalam kolom ini
df_new['gender'].value_counts()

F      14164
M       7259
XNA        1
Name: gender, dtype: int64

Terdapat nilai XNA dan hanya 1 data, oleh karena itu lebih baik dihapus.

In [39]:
# Atasi nilai-nilai yang bermasalah, jika ada
df_new.drop(df_new[df_new['gender'] == 'XNA'].index, inplace=True)

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [40]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
df_new['gender'].value_counts()

F    14164
M     7259
Name: gender, dtype: int64

In [41]:
# Mari kita lihat nilai dalam kolom ini
df_new['income_type'].value_counts()

employee                       11064
business                        5064
retiree                         3836
civil servant                   1453
entrepreneur                       2
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [42]:
# Periksa duplikat
df_new.duplicated().sum()


71

In [43]:
# Atasi duplikat, jika ada
df_new.drop_duplicates(inplace=True)

/tmp/ipykernel_77/3413823760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.drop_duplicates(inplace=True)


In [44]:
# Lakukan pemeriksaan terakhir untuk mengecek apakah kita memiliki duplikat
df_new.duplicated().sum()

0

In [45]:
# Periksa ukuran dataset yang sekarang Anda miliki setelah manipulasi pertama yang Anda lakukan
df_new.shape

(21352, 12)

Sebelumnya data memiliki ukuran (21525, 12), setelah menghapus duplikatnya dataset memiliki ukuran (21352, 12).


### Bekerja dengan nilai yang hilang

In [46]:
# Temukan dictionary
education_dictionary = dict(zip(df_new.education, df_new.education_id))
education_dictionary

{"bachelor's degree": 0,
 'secondary education': 1,
 'some college': 2,
 'primary education': 3,
 'graduate degree': 4}

In [47]:
fam_dictionary = dict(zip(df_new.family_status, df_new.family_status_id))
fam_dictionary

{'married': 0,
 'civil partnership': 1,
 'widow / widower': 2,
 'divorced': 3,
 'unmarried': 4}

### Memperbaiki nilai yang hilang 

Kali ini, kita akan mengatasi nilai yang hilang pada kolom `total_income`. Untuk itu, kita perlu mengkategorikan data usia karena income diperkirakan dipengaruhi oleh usia nasabah. Setelah mengkategorikan data, kita akan melihat mean dan median dari data untuk melihat apa yang akan digunakan untuk mengisi nilai yang hilang.

In [48]:
# Mari kita tulis sebuah fungsi untuk menghitung kategori usia
def years_grouped(age):    
    if age <= 25:
        return '<=25'
    if 26 <= age <= 50:
        return '26 - 50'
    if age >= 51:
        return '>=51'

In [49]:
# Lakukan pengujian untuk melihat apakah fungsi Anda bekerja atau tidak
years_grouped(42)

'26 - 50'

In [50]:
# Buatlah kolom baru berdasarkan fungsi
df_new['years_grouped'] = df_new['dob_years'].apply(years_grouped)

/tmp/ipykernel_77/2264452651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['years_grouped'] = df_new['dob_years'].apply(years_grouped)


In [51]:
# Periksa bagaimana nilai di dalam kolom baru
df_new['years_grouped']

0        26 - 50
1        26 - 50
2        26 - 50
3        26 - 50
4           >=51
          ...   
21520    26 - 50
21521       >=51
21522    26 - 50
21523    26 - 50
21524    26 - 50
Name: years_grouped, Length: 21352, dtype: object

In [52]:
df_new.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21352.000000,19259.000000,21352.000000,21352.000000,21352.000000,21352.000000,19259.000000
mean,0.476115,66905.789237,43.476817,0.817722,0.972649,0.081163,26794.133121
std,0.758627,139025.028132,12.241877,0.548717,1.421020,0.273092,16494.347322
min,-1.000000,24.141633,19.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,926.823974,33.000000,1.000000,0.000000,0.000000,16494.864000
50%,0.000000,2197.320350,43.000000,1.000000,0.000000,0.000000,23200.877000
75%,1.000000,5540.399763,53.000000,1.000000,1.000000,0.000000,32536.052000
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [53]:
# Buat tabel tanpa nilai yang hilang dan tampilkan beberapa barisnya untuk memastikan semuanya berjalan dengan baik
df_notna = df_new.dropna()
df_notna

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_grouped
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,26 - 50
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,26 - 50
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26 - 50
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26 - 50
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,>=51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,26 - 50
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,>=51
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,26 - 50
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,26 - 50


In [54]:
# Perhatikan nilai rata-rata untuk pendapatan berdasarkan faktor yang telah Anda identifikasi
total_inc_mean = df_notna.pivot_table(index='years_grouped', values='total_income', aggfunc='mean')
total_inc_mean

,total_income
years_grouped,
26 - 50,28124.372580
<=25,23447.795802
>=51,24704.150073


In [55]:
# Perhatikan nilai median untuk pendapatan berdasarkan faktor yang telah Anda identifikasi
total_inc_med = df_notna.pivot_table(index='years_grouped', values='total_income', aggfunc='median')
total_inc_med

,total_income
years_grouped,
26 - 50,24533.0120
<=25,21423.8355
>=51,21310.9445


Nilai median akan digunakan untuk mengisi nilai yang hilang karena lebih menggambarkan persebaran data dibandingkan dengan mean.

In [56]:
#  Tulis fungsi yang akan kita gunakan untuk mengisi nilai yang hilang    

def fill_median_total_inc(dataframe, agg_column, value_column) :
    grouped_values = dataframe.groupby(agg_column)[value_column].median().reset_index()
    size = len(grouped_values)
    for i in range(size):
        group = grouped_values[agg_column][i]
        value = grouped_values[value_column][i]
        dataframe.loc[(dataframe[agg_column]==group) & (dataframe[value_column].isna()), value_column] = value
    return dataframe

In [57]:
# Memeriksa bagaimana nilai di dalam kolom baru
fill_median_total_inc(dataframe = df_new, agg_column = 'years_grouped', value_column = 'total_income')

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_grouped
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,26 - 50
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,26 - 50
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26 - 50
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26 - 50
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,>=51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,26 - 50
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,>=51
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,26 - 50
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,26 - 50


In [58]:
# Terapkan fungsi tersebut ke setiap baris
df_new = fill_median_total_inc(dataframe = df_new, agg_column = 'years_grouped', value_column = 'total_income')

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [59]:
# Periksa apakah kita mendapatkan kesalahan
df_new['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21520    35966.698
21521    24959.969
21522    14347.610
21523    39054.888
21524    13127.587
Name: total_income, Length: 21352, dtype: float64

In [60]:
df_new.isna().sum()

children               0
days_employed       2093
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
years_grouped          0
dtype: int64

Kali ini, kita akan mengatasi nilai yang hilang pada kolom `days_employed`. Untuk itu, kita perlu melihat mean dan median dari data untuk melihat apa yang akan digunakan untuk mengisi nilai yang hilang.

In [61]:
# Distribusi median dari `days_employed` berdasarkan parameter yang Anda identifikasi
days_emp_med = df_notna.pivot_table(index='income_type', values='days_employed', aggfunc='median')
days_emp_med

,days_employed
income_type,
business,1548.009883
civil servant,2673.404956
employee,1576.067689
entrepreneur,520.848083
paternity / maternity leave,3296.759962
retiree,365176.336775
student,578.751554
unemployed,366413.652744


In [62]:
# Distribusi rata-rata dari `days_employed` berdasarkan parameter yang Anda identifikasi
days_emp_mean = df_notna.pivot_table(index='income_type', values='days_employed', aggfunc='mean')
days_emp_mean 

,days_employed
income_type,
business,2112.744402
civil servant,3388.508552
employee,2328.603723
entrepreneur,520.848083
paternity / maternity leave,3296.759962
retiree,365015.727554
student,578.751554
unemployed,366413.652744


Untuk mengisi nilai yang hilang pada kolom `days_employed` menggunakan nilai median atau nilai tengah karena nilai mean atau rata-ratanya terlalu tinggi sehingga kurang menggambarkan keseluruhan dataset.

In [63]:
# Mari tulis fungsi yang menghitung rata-rata atau median (tergantung keputusan Anda) berdasarkan parameter yang Anda identifikasi
def fill_median_days_emp(dataframe, agg_column, value_column) :
    grouped_values = dataframe.groupby(agg_column)[value_column].median().reset_index()
    size = len(grouped_values)
    for i in range(size):
        group = grouped_values[agg_column][i]
        value = grouped_values[value_column][i]
        dataframe.loc[(dataframe[agg_column]==group) & (dataframe[value_column].isna()), value_column] = value
    return dataframe

In [64]:
# Periksa apakah fungsi Anda dapat bekerja
fill_median_days_emp(dataframe = df_new, agg_column = 'income_type', value_column = 'days_employed')

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_grouped
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,26 - 50
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,26 - 50
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26 - 50
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26 - 50
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,>=51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,26 - 50
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,>=51
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,26 - 50
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,26 - 50


In [65]:
# Terapkan fungsi ke days_employed
df_new = fill_median_days_emp(dataframe = df_new, agg_column = 'income_type', value_column = 'days_employed')


/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [66]:
df_new['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21352, dtype: float64

In [67]:
# Periksa entri di semua kolom - pastikan kita memperbaiki semua nilai yang hilang
df_new.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
years_grouped       0
dtype: int64

In [68]:
# Tampilkan nilai data yang Anda pilih untuk pengkategorian
df_new['purpose']

0          purchase of the house
1                   car purchase
2          purchase of the house
3        supplementary education
4              to have a wedding
                  ...           
21520       housing transactions
21521          purchase of a car
21522                   property
21523          buying my own car
21524               to buy a car
Name: purpose, Length: 21352, dtype: object

In [69]:
# Periksa nilai unik
df_new['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [70]:
# Mari kita tulis sebuah fungsi untuk mengategorikan data berdasarkan topik umum
def general_purpose(row):
    if 'car' in row: 
        return 'car'
    if 'property' in row or 'real estate' in row or 'renovation' in row or 'hous' in row:
        return 'house'
    if 'university'in row or 'educated' in row or 'education' in row:
        return 'education'
    if 'wedding' in row: 
        return 'wedding'
    else:
        'None'

In [71]:
# Buat kolom yang memuat kategori dan hitung nilainya
df_new['general_purpose'] = df_new['purpose'].apply(general_purpose)

/tmp/ipykernel_77/1338101129.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['general_purpose'] = df_new['purpose'].apply(general_purpose)


In [72]:
# Lihat semua data numerik di kolom yang Anda pilih untuk pengkategorian
df_new['general_purpose'].value_counts()

house        10763
car           4284
education     3995
wedding       2310
Name: general_purpose, dtype: int64

In [73]:
# Dapatkan kesimpulan statistik untuk kolomnya
df_new.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21352.000000,21352.000000,21352.000000,21352.000000,21352.000000,21352.000000,21352.000000
mean,0.476115,67030.735551,43.476817,0.817722,0.972649,0.081163,26453.453465
std,0.758627,139176.471187,12.241877,0.548717,1.421020,0.273092,15706.643805
min,-1.000000,24.141633,19.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,1023.688788,33.000000,1.000000,0.000000,0.000000,17223.821250
50%,0.000000,1997.955280,43.000000,1.000000,0.000000,0.000000,23576.204500
75%,1.000000,5321.001947,53.000000,1.000000,1.000000,0.000000,31321.653000
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [74]:
df_new['total_income'].describe()

count     21352.000000
mean      26453.453465
std       15706.643805
min        3306.762000
25%       17223.821250
50%       23576.204500
75%       31321.653000
max      362496.645000
Name: total_income, dtype: float64

In [75]:
# Buat fungsi yang melakukan pengkategorian menjadi kelompok numerik yang berbeda berdasarkan rentang
def range_total_inc(total_income):
    if total_income < 20000:
        return 'low'
    if 20000 <= total_income <= 25000:
        return 'medium'
    if 25000 <= total_income <= 35000:
        return 'high'
    else:
        return 'very high'
    return 'None'

In [76]:
# Buat kolom yang memuat kategori
df_new['range_total_inc'] = df_new['total_income'].apply(range_total_inc)
df_new['range_total_inc'].value_counts()

/tmp/ipykernel_77/1102842488.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['range_total_inc'] = df_new['total_income'].apply(range_total_inc)


low          7332
medium       5461
high         4511
very high    4048
Name: range_total_inc, dtype: int64

In [77]:
# Hitung setiap nilai kategori untuk melihat pendistribusiannya
df_new

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_grouped,general_purpose,range_total_inc
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,26 - 50,house,very high
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,26 - 50,car,low
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26 - 50,house,medium
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26 - 50,education,very high
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,>=51,wedding,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,26 - 50,house,very high
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,>=51,car,medium
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,26 - 50,house,low
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,26 - 50,car,very high


In [78]:
def child(children):
    if children < 1:
        return 'No'
    return 'Yes'

In [79]:
df_new['child'] = df_new['children'].apply(child)
df_new['child'].value_counts()

/tmp/ipykernel_77/2789945428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['child'] = df_new['children'].apply(child)


No     14068
Yes     7284
Name: child, dtype: int64

## Tahap 3: Memeriksa hipotesis


**Apakah terdapat korelasi antara memiliki anak dengan probabilitas melakukan gagal bayar pinjaman?**

In [80]:
# Periksa data anak dan data gagal bayar pinjaman
df_new.pivot_table(index= ['children','debt'], values= 'days_employed', aggfunc= 'count')


days_employed
children debt               
-1       0                46
         1                 1
 0       0             12963
         1              1058
 1       0              4351
         1               441
 2       0              1912
         1               202
 3       0               301
         1                27
 4       0                37
         1                 4
 5       0                 9

In [81]:
# Hitung persentase gagal bayar berdasarkan jumlah anak
# percent_debt_true= df_new[df_new['debt'] == 1].count() / df_new['debt'].count()*100
# percent_debt_true
pivot_table_children = df_new.pivot_table(index='child', columns= 'debt', values='dob_years', aggfunc='count')
pivot_table_children['percentage_debt_true'] = pivot_table_children[1] / (pivot_table_children[1] + pivot_table_children[0]) * 100
pivot_table_children

debt,0,1,percentage_debt_true
child,,,
No,13009,1059,7.527722
Yes,6610,674,9.253158


In [82]:
pivot_table_children['percentage_debt_false'] = pivot_table_children[0] / (pivot_table_children[1] + pivot_table_children[0]) * 100
pivot_table_children

debt,0,1,percentage_debt_true,percentage_debt_false
child,,,,
No,13009,1059,7.527722,92.472278
Yes,6610,674,9.253158,90.746842


**Kesimpulan**

* Sebanyak 7% nasabah yang tidak memiliki anak gagal melakukan pembayaran
* Nasabah yang memiliki anak memiliki presentase gagal bayar sebesar 9%

**Apakah terdapat korelasi antara status keluarga dengan probabilitas melakukan gagal bayar pinjaman?**

In [83]:
# Periksa data status keluarga dan data gagal bayar pinjaman
pivot_debt_fam = df_new.pivot_table(index=['family_status','debt'], values='days_employed', aggfunc='count')
pivot_debt_fam

days_employed
family_status     debt               
civil partnership 0              3743
                  1               386
divorced          0              1100
                  1                85
married           0             11363
                  1               927
unmarried         0              2521
                  1               273
widow / widower   0               892
                  1                62

In [84]:
# Hitung persentase gagal bayar berdasarkan status keluarga
pivot_debt_fam = df_new.pivot_table(index='family_status', columns= 'debt', values='days_employed', aggfunc='count')
pivot_debt_fam['percentage_debt_true'] = pivot_debt_fam[1] / (pivot_debt_fam[1] + pivot_debt_fam[0]) * 100
pivot_debt_fam['percentage_debt_false'] = pivot_debt_fam[0] / (pivot_debt_fam[1] + pivot_debt_fam[0]) * 100
pivot_debt_fam

debt,0,1,percentage_debt_true,percentage_debt_false
family_status,,,,
civil partnership,3743,386,9.348511,90.651489
divorced,1100,85,7.172996,92.827004
married,11363,927,7.542718,92.457282
unmarried,2521,273,9.770938,90.229062
widow / widower,892,62,6.498952,93.501048


**Kesimpulan**

* Presentase gagal bayar tertinggi dimiliki oleh nasabah yang belum menikah yaitu hampir 10%
* Sebanyak 6% nasabah yang cerai mati gagal melakukan pembayaran
* Status lain seperti bercerai, partner, dan menikah memiliki presentase gagal bayar 7 - 9%

**Apakah terdapat korelasi antara tingkat pendapatan dengan probabilitas melakukan gagal bayar pinjaman?**

In [85]:
# Periksa data tingkat pendapatan dan data gagal bayar pinjaman
df_new.pivot_table(index=['range_total_inc','debt'], values='days_employed', aggfunc='count')

days_employed
range_total_inc debt               
high            0              4128
                1               383
low             0              6727
                1               605
medium          0              5005
                1               456
very high       0              3759
                1               289

In [86]:
# Hitung persentase gagal bayar berdasarkan tingkat pendapatan
pivot_debt_inc = df_new.pivot_table(index='range_total_inc', columns= 'debt', values='days_employed', aggfunc='count')
pivot_debt_inc['percentage_debt_true'] = pivot_debt_inc[1] / (pivot_debt_inc[1] + pivot_debt_inc[0]) * 100
pivot_debt_inc['percentage_debt_false'] = pivot_debt_inc[0] / (pivot_debt_inc[1] + pivot_debt_inc[0]) * 100
pivot_debt_inc

debt,0,1,percentage_debt_true,percentage_debt_false
range_total_inc,,,,
high,4128,383,8.490357,91.509643
low,6727,605,8.251500,91.748500
medium,5005,456,8.350119,91.649881
very high,3759,289,7.139328,92.860672


**Kesimpulan**

* Berdasarkan rentang total pendapatan, presentase gagal bayar tertinggi justru yang berpenghasilan 'high'. Namun selisih presentase high, low, dan medium hanya sedikit yaitu berada di kisaran 8%
* Nasabah yang memiliki presentase gagal bayar terendah yaitu kategori 'very high' sebesar 7%

**Bagaimana tujuan kredit memengaruhi persentase gagal bayar?**

In [87]:
# Periksa persentase tingkat gagal bayar untuk setiap tujuan kredit dan lakukan penganalisisan
df_new.pivot_table(index=['general_purpose','debt'], values='days_employed', aggfunc='count')

days_employed
general_purpose debt               
car             0              3884
                1               400
education       0              3625
                1               370
house           0              9984
                1               779
wedding         0              2126
                1               184

In [88]:
pivot_debt_purpose = df_new.pivot_table(index='general_purpose', columns= 'debt', values='days_employed', aggfunc='count')
pivot_debt_purpose['percentage_debt_true'] = pivot_debt_purpose[1] / (pivot_debt_purpose[1] + pivot_debt_purpose[0]) * 100
pivot_debt_purpose['percentage_debt_false'] = pivot_debt_purpose[0] / (pivot_debt_purpose[1] + pivot_debt_purpose[0]) * 100
pivot_debt_purpose

debt,0,1,percentage_debt_true,percentage_debt_false
general_purpose,,,,
car,3884,400,9.337068,90.662932
education,3625,370,9.261577,90.738423
house,9984,779,7.237759,92.762241
wedding,2126,184,7.965368,92.034632


**Kesimpulan**

* Sebanyak 9% nasabah yang bertujuan melakukan pinjaman untuk biaya kendaraan dan pendidikan gagal melakukan pembayaran
* Presentase gagal bayar terendah yaitu berada pada kategori biaya untuk rumah sebesar 7%

# Kesimpulan umum 

1. Dataset memiliki nilai yang harus diperbaiki karena terdapat nilai yang hilang, nilai yang terduplikat, nilai yang kurang logis(usia yang bernilai 0, jumlah anak minus dan puluhan, gender yang bernilai XNA)
2. Kesalahan yang ada dalam dataset dapat disebabkan oleh adanya kesalahan dalam penginputan data
3. Untuk nilai yang terduplikat diatasi dengan drop duplicate
4. Usia yang bernilai 0 tidak diikutsertakan dalam analisis gagal bayar karena akan sulit apakah itu kesalahan dalam penginputan data atau bukan,
5. Jumlah anak yang bernilai minus diatasi dengan abs() dan nilai yang puluhan diperbaiki dengan menghapus angka 0 dibelakangnya karena diasumsikan terdapat kesalahan penginputan data.
6. Gender yang bernilai XNA dihapus karena sulit diketahui apa nilai aslinya dan data tersebut hanya ada satu.
7. Nilai yang hilang pada dataset berada pada kolom total_income dan days_employed bersifat acak atau tidak memiliki pola tertentu. Median akan digunakan untuk mengisi data yang hilang karena lebih menggambarkan persebaran data.
8. Nilai yang hilang pada kolom total_income diisi dengan mengkategorikan usia nasabah dan mengisinya dengan median
9. Nilai yang hilang pada kolom days_employed diisi dengan mengisi nilai mediannya.
10. Dari hasil analisis data diperoleh bahwa **probabilitas nasabah terhadap status keluarga, anak, tingkat pendidikan, tujuan peminjaman, dan pendapatannya tidak mempengaruhi gagal bayarnya**. 
11. Nasabah yang mengalami **gagal bayar** di setiap kategori **hanya 6 - 9%** dan sisanya yaitu sekitar 90% berhasil melakukan pembayaran dengan persebaran nilai yang cukup merata.

